In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Colab Notebooks'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
!pip uninstall scikit-learn
!pip install scikit-learn==1.2.2

Found existing installation: scikit-learn 1.5.1
Uninstalling scikit-learn-1.5.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/scikit_learn-1.5.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/sklearn/*
Proceed (Y/n)? y
  Successfully uninstalled scikit-learn-1.5.1
y
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires scikit-learn>=1.4.2, but you have scikit-learn 1.2.2 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from keras import Sequential
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from keras import layers, Input
from sklearn.model_selection import GridSearchCV
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv1D, MaxPooling1D

In [ ]:
dataset = pd.read_csv('healthcare-dataset-stroke-data.csv')
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


Xử lý dữ liệu missing

In [ ]:
dataset['bmi'].fillna(dataset['bmi'].mean(), inplace=True)
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1


Xóa cột id

In [ ]:
dataset.drop(columns=['id'],inplace=True)
dataset.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1


Loại bỏ hàng có gender là "other"

In [ ]:
#trước khi xử lý
dataset['gender'].value_counts()

gender
Female    2994
Male      2115
Other        1
Name: count, dtype: int64

In [ ]:
# Sau khi xử lý
dataset = dataset[dataset['gender'] != 'Other']
dataset['gender'].value_counts()

gender
Female    2994
Male      2115
Name: count, dtype: int64

Chuyển dữ liệu cột gender, ever_maried và Residence_type thành binary

In [ ]:
dataset['gender'] = dataset['gender'].map({'Female':1 ,'Male':0})
dataset['ever_married'] = dataset['ever_married'].map({'Yes':1 ,'No':0})
dataset['Residence_type'] = dataset['Residence_type'].map({'Rural':1 ,'Urban':0})
dataset.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,1,Private,0,228.69,36.600000,formerly smoked,1
1,1,61.0,0,0,1,Self-employed,1,202.21,28.893237,never smoked,1
2,0,80.0,0,1,1,Private,1,105.92,32.500000,never smoked,1
3,1,49.0,0,0,1,Private,0,171.23,34.400000,smokes,1
4,1,79.0,1,0,1,Self-employed,1,174.12,24.000000,never smoked,1


chuyển các cột còn lại thành các nhị phân

In [ ]:
dataset = pd.get_dummies(dataset).astype(int)
dataset.head()

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,67,0,1,1,0,228,36,1,0,0,1,0,0,0,1,0,0
1,1,61,0,0,1,1,202,28,1,0,0,0,1,0,0,0,1,0
2,0,80,0,1,1,1,105,32,1,0,0,1,0,0,0,0,1,0
3,1,49,0,0,1,0,171,34,1,0,0,1,0,0,0,0,0,1
4,1,79,1,0,1,1,174,24,1,0,0,0,1,0,0,0,1,0


xử lý imbalanced

In [ ]:
X = np.array(dataset.drop(columns=['stroke']))
y = np.array(dataset['stroke'])

In [ ]:
from collections import Counter
print('Original dataset shape %s' % Counter(y))
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_resampled))

Original dataset shape Counter({0: 4860, 1: 249})
Resampled dataset shape Counter({1: 4860, 0: 4860})


In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
rf_model = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'max_features': ['auto', 'sqrt']
}
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,  n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
best_params_rf = grid_search.best_params_
print(f'Best parameters found: {best_params_rf}')

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found: {'max_depth': 10, 'max_features': 'auto', 'n_estimators': 200}


In [ ]:
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')
print(f"Random Forest - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Random Forest - Accuracy: 0.9510763209393346, Precision: 0.9510763209393346, Recall: 0.9510763209393346, F1 Score: 0.9510763209393346


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

optimizer = ['adam', 'rmsprop']
batch_size = [10, 20, 40]
epochs =  [30, 50, 100]
param_grid_nn = {"optimizer":optimizer, "batch_size":batch_size, "epochs":epochs}

keras_model = KerasClassifier(build_fn= model, verbose=0)
grid_search_nn = GridSearchCV(estimator=keras_model, param_grid=param_grid_nn, n_jobs=-1, verbose=1)
grid_search_nn.fit(X_train, y_train)

best_params = grid_search_nn.best_params_
print(f"Best parameters found: {best_params}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Best parameters found: {'batch_size': 40, 'epochs': 30, 'optimizer': 'adam'}


In [ ]:
best_nn = grid_search_nn.best_estimator_
y_pred_nn = (best_nn.predict(X_test) > 0.5).astype("int32")
accuracy_nn = accuracy_score(y_test, y_pred_nn)
precision_nn = precision_score(y_test, y_pred_nn, average='micro')
recall_nn = recall_score(y_test, y_pred_nn,  average='micro')
f1_nn = f1_score(y_test, y_pred_nn,  average='micro')

print(f"Neural Network - Accuracy: {accuracy_nn}, Precision: {precision_nn}, Recall: {recall_nn}, F1 Score: {f1_nn}")

Neural Network - Accuracy: 0.9497716894977168, Precision: 0.9497716894977168, Recall: 0.9497716894977168, F1 Score: 0.9497716894977168


So sánh kết quả

In [ ]:
from prettytable import PrettyTable
t = PrettyTable(['Algorithms', 'Accuracy', 'Precision', 'Recall', 'F1 Score' ])
t.add_row(['Random Forest', accuracy, precision, recall, f1])
t.add_row(['Neural Network', accuracy_nn, precision_nn, recall_nn, f1_nn])
print(t)

+----------------+--------------------+--------------------+--------------------+--------------------+
|   Algorithms   |      Accuracy      |     Precision      |       Recall       |      F1 Score      |
+----------------+--------------------+--------------------+--------------------+--------------------+
| Random Forest  | 0.9510763209393346 | 0.9510763209393346 | 0.9510763209393346 | 0.9510763209393346 |
| Neural Network | 0.9497716894977168 | 0.9497716894977168 | 0.9497716894977168 | 0.9497716894977168 |
+----------------+--------------------+--------------------+--------------------+--------------------+
